#TP BLOC 6 : application des réseaux récurrents à la traduction de chaines de caractères

Objectif : des prénoms, représentés sous la forme de chaines de caractères, ont été encryptés par un algorithme inconnu.

Il vous est donné un ensemble de prénoms "d'entrainement" pour lesquels vous disposerez de la chaine de caractère cryptée et de la chaine de caractères décryptée.

Vous devrez apprendre un modèle à base de LSTM capable de décrypter un jeu de prénoms de test pour lequel seul la version cryptée est fournie. Pour les besoins du TP, les chaines de caractères décryptées vous sont également fournies pour le jeu de test afin que vous puissiez vérifier le bon fonctionnement de votre modèle.

---



---



### 1ʳᵉ étape (google colab uniquement)

Si vous souhaitez utiliser google colab, la cellule de code qui suit vous permettra d'accéder à vos fichiers qui se trouvent sur google drive.
Si vous travaillez en local sur votre machine, vous pouvez ignorer cette cellule.**bold text**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
cur_dir = 'gdrive/My Drive/Colab Notebooks/formation IA niveau 3/module 7 advanced ML/Day2'
# !ls 'gdrive/My Drive/Colab Notebooks/formation IA niveau 3/module 7 advanced ML/Day2'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 2nd étape : récupération des données

Les séquences de chaines de caractères sont contenus dans deux fichiers, l'un avec les données d'entrainement, l'autre avec celles de test.

Pour chacun, deux tenseurs sont fournis : le premier contient les séquences 'source' qui sont les séquences cryptées, l'autre les séquences 'target' qui sont décryptés. 

Nous vous donnons également une fonction qui permet de transformer les séquences de chiffres en séquences de lettres, plus facile à lire pour les humains.

Toutes les séquences ont été ramenées à 16 caractères par padding. Vous remarquerez aussi qu'il y a un caractère spécial qui marque le début de la chaine.

En regardant les paires de séquences cryptées/décryptées, essayez de comprendre quel code est utilisé pour le cryptage.

En quoi cela justifie-t-il l'usage de réseaux de neurones prenant en compte des données séquentielles ?

In [ ]:
import torch

src_train,tgt_train,num_codes = torch.load(cur_dir+'/'+'train_set.pt')
src_test,tgt_test,num_codes = torch.load(cur_dir+'/'+'test_set.pt')
seq_len = tgt_train.shape[1]


print("taille des tenseurs pour l'entrainement", tgt_train.shape,src_train.shape)
print("taille des tenseurs pour le test", tgt_test.shape,src_test.shape)
print("longueur des séquences : ",seq_len)
def ascii2str(x):
  string =  ''.join([chr(int(i)+ord('a')) if (int(i)+ord('a'))!=ord('z')+2 else '.'  for i in x])
  new_string = string.replace("{", "-" )
  return new_string

for i in range(20):
  print('source : [',ascii2str(src_train[i,:]),"] -> target : [",ascii2str(tgt_train[i,:]),']')



taille des tenseurs pour l'entrainement torch.Size([2453, 16]) torch.Size([2453, 16])
taille des tenseurs pour le test torch.Size([614, 16]) torch.Size([614, 16])
longueur des séquences :  16
source : [ .tdqdefghijklmno ] -> target : [ .sam------------ ]
source : [ .udvbtfghijklmno ] -> target : [ .taryn---------- ]
source : [ .srfoughijklmnop ] -> target : [ .robin---------- ]
source : [ .tjmqvphijklmnop ] -> target : [ .shiloh--------- ]
source : [ .nhpguehijklmnop ] -> target : [ .melany--------- ]
source : [ .gtbhdxqlijklmno ] -> target : [ .fr-d-ric------- ]
source : [ .srwgsnpzlmnopqr ] -> target : [ .rosaleen------- ]
source : [ .ehttfghijklmnop ] -> target : [ .deon----------- ]
source : [ .mrzltulklmnopqr ] -> target : [ .louella-------- ]
source : [ .mrvwpnijklmnopq ] -> target : [ .lorrie--------- ]
source : [ .ndvowwijklmnopq ] -> target : [ .marion--------- ]
source : [ .xhriltuijklmnop ] -> target : [ .wendell-------- ]
source : [ .taefy-rpz-topqr ] -> target : [ .s-basti

### 3eme étape : construction d'un dataset pytorch

Vous allez désormais construire un objet de type 'torch.utils.data.Dataset' dataset compatible avec les "data loader" (torch.utils.data.DataLoader) de pytorch. 

Les dernières instructions de la cellule (celles qui vous sont fournies) vous permettra de vérifier que ce que vous avez fait fonctionne.

In [ ]:
"""
à vous d'écrire cette classe

class CustomDataset(torch.utils.data.Dataset):
...
"""


trainset = CustomDataset(src_train,tgt_train)
testset = CustomDataset(src_test,tgt_test)

train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=16, shuffle=True)

for x_src, x_tgt in test_loader:
  for i in range(x_src.shape[0]):
    print(ascii2str(x_src[i]), '->', ascii2str(x_tgt[i]))
  break

.coaaefghijklmno -> .alyx-----------
.cokkyvyyklmnopq -> .algernon-------
.khqdefghijklmno -> .jem------------
.klqdefghijklmno -> .jim------------
.htehpnijklmnopq -> .gracie---------
.shro-jijklmnopq -> .renita---------
.mrzkfghijklmnop -> .love-----------
.kxpoiwxqlmnopqr -> .julianne-------
.trvwlthijklmnop -> .sorrel---------
.mlpqcfghijklmno -> .lilly----------
.kqnmwhijklmnopq -> .inigo----------
.tdputghijklmnop -> .salom----------
.gohwllkjklmnopq -> .eldreda--------
.urqrcfghijklmno -> .tommy----------
.hhv-h-ojklmnopq -> .gervase--------
.mdpqhghijklmnop -> .lalla----------


### 4ᵉ étape : construction du modèle à base de LSTM

Vous construirez ensuite le modèle de LSTM qui permet de faire la traduction de chaines de *caractères*

In [ ]:
"""
à vous d'écrire le modèle

class mymodel(torch.nn.Module):


"""


### 5eme étape : entrainement du modèle

Vous pourrez, par exemple, représenter les caractères par un embedding de taille 20.

In [ ]:
"""

à vous d'entrainer le modèle


"""

epoch:  0 loss:  74.02486073970795
exemple de décodage d'une donnée de train :  .lexie----------  ->  .a-------------- <fin>
epoch:  10 loss:  6.972810223698616
exemple de décodage d'une donnée de train :  .selima---------  ->  .selina--------- <fin>
epoch:  20 loss:  2.3015378937125206
exemple de décodage d'une donnée de train :  .percy----------  ->  .percy---------- <fin>
epoch:  30 loss:  1.2281909212470055
exemple de décodage d'une donnée de train :  .chad-----------  ->  .chad----------- <fin>
epoch:  40 loss:  0.8950350396335125
exemple de décodage d'une donnée de train :  .lauryn---------  ->  .lauryn--------- <fin>


### 6eme étape : décodage du jeu de test



In [ ]:
"""

à vous de faire les inférences sur le jeu de test

"""

décodage de [ .daojy-nijklmnop ] -> [ .c-leste-------- ] GT =  .c-leste--------
décodage de [ .md-gulkjklmnopq ] -> [ .lawanda-------- ] GT =  .lawanda--------
décodage de [ .qj.pqplkjklmnop ] -> [ .ohyllida------- ] GT =  .phyllida-------
décodage de [ .x-pttkghijklmno ] -> [ .wylond--------- ] GT =  .wymond---------
décodage de [ .cpcqpnijklmnopq ] -> [ .am-lie--------- ] GT =  .am-lie---------
décodage de [ .thvllghijklmnop ] -> [ .serge---------- ] GT =  .serge----------
décodage de [ .xlpkefghijklmno ] -> [ .wilf----------- ] GT =  .wilf-----------
décodage de [ .woaxyihijklmnop ] -> [ .vlyssa--------- ] GT =  .ulyssa---------
décodage de [ .tje.tkjijklmnop ] -> [ .shawnda-------- ] GT =  .shawnda--------
décodage de [ .cupounijklmnopq ] -> [ .arline--------- ] GT =  .arline---------
décodage de [ .kojhvhijklmnopq ] -> [ .jlean---------- ] GT =  .ilean----------
décodage de [ .djunyzrpklmnopq ] -> [ .chrissie------- ] GT =  .chrissie-------
décodage de [ .clpqcfghijklmno ] -> [ .a